In [39]:
import numpy as np
from collections import Counter
import Sudoku
import math
import random


def available_numbers(master, taken):
    master_arr = np.array([i for i in range(1, 10) for _ in range(9)])
    clue_arr = taken
    clueCount = Counter(clue_arr)
    fullCount = Counter(master_arr)
    remainCount = fullCount - clueCount
    result = np.array([val for val, cnt in remainCount.items() for _ in range(cnt)])
    return result
    

class SimulatedAnnealing:
    def __init__(self, sudoku_init, t_sched = None, t_init = 10):
        self.sudoku = sudoku_init
        self.grid = np.array(sudoku_init.board)
        self.fixed = np.array(sudoku_init.fixed)
        self.length = sudoku_init.length
        self.can_swap = ~self.fixed
        self.num_spots = self.can_swap.sum()
        self.possible_numbers = available_numbers(np.array([i for i in range(1, 10) for _ in range(9)]), self.grid.flatten())
        self.populate()
        
        self.T_init = t_init
        self.T = self.T_init
        self.iters = 1
        if t_sched==None:
            self.decay = 0.9999
        else:
            self.decay = t_sched
        self.error_count = 99999999
        self.T_min = 0.0000001
        
        #for testing
        self.num_goodswaps = 0
        self.num_badswaps = 0
        self.num_rejectedswaps=0
        
        
        
    def populate(self):
        for box_row in range(3):
            for box_col in range(3):
                numbers = set(range(1,10))
                for r in range(3*box_row, 3*box_row+3):
                    for c in range(3*box_col, 3*box_col+3):
                        if self.fixed[r][c]:
                            numbers.discard(self.grid[r][c])
                for r in range(3*box_row, 3*box_row+3):
                    for c in range(3*box_col, 3*box_col+3):
                        if not self.fixed[r][c]:
                            self.grid[r][c] = numbers.pop()
                        
    def swap(self):
        # --- 1. Choose a random 3×3 subgrid ---
        box = random.randint(0, 8)
        row_start, col_start = 3 * (box // 3), 3 * (box % 3)

        # --- 2. Collect all non-fixed positions in this subgrid ---
        candidates = [
            (r, c)
            for r in range(row_start, row_start + 3)
            for c in range(col_start, col_start + 3)
            if not self.fixed[r][c]
        ]

        # If fewer than two swappable cells exist, pick a new box
        if len(candidates) < 2:
            return self.swap()  # recursive retry

        # --- 3. Pick two random positions and swap them ---
        (r1, c1), (r2, c2) = random.sample(candidates, 2)
        self.grid[r1][c1], self.grid[r2][c2] = self.grid[r2][c2], self.grid[r1][c1]

        # Compute error delta
        currErr = self.sudoku.numErrors()
        newErr = self.sudoku.numErrors(grid=self.grid.tolist())

        # Acceptance rule
        delta = newErr - currErr
        if delta < 0:  # improvement
            self.sudoku.board = self.grid.tolist()
            self.error_count = newErr
            self.num_goodswaps += 1
        else:
            # probability of accepting worse move
            prob = math.exp(-delta / self.T)
            if random.random() < prob:
                self.sudoku.board = self.grid.tolist()
                self.error_count = newErr
                self.num_badswaps += 1
            else:
                # revert swap
                self.grid[r1][c1], self.grid[r2][c2] = self.grid[r2][c2], self.grid[r1][c1]
                self.num_rejectedswaps += 1

        # Cool down
        self.iters += 1
        self.T *= self.decay

        
    def solve(self, display=False):
        while self.error_count>0:
            self.swap()
            if self.iters % 1000 ==0 and display:
                print('Current Iteration: ', self.iters, '; Current errors: ', self.error_count, '; Current temperature: ', self.T)
                print('Good swaps: ', self.num_goodswaps, '; Bad swaps: ', self.num_badswaps, '; Rejected swaps: ', self.num_rejectedswaps)
                self.num_goodswaps = 0
                self.num_badswaps = 0
                self.num_rejectedswaps=0
            if self.iters % 20000 == 0:
                self.T *= 1.05  # small “reheat”
            if self.iters % 60000 == 0: #Restart if stuck
                self.T = 5
        #print('final error count: ', self.error_count)
        print('number of iterations: ', self.iters)
        #print('final grid: ', self.grid)
        return self.iters

                 

In [40]:
import Sudoku

board1 = Sudoku.Sudoku(3)
digitString = "070000043040009610800634900094052000358460020000800530080070091902100005007040802"
board1.fillFromString(digitString)
x = SimulatedAnnealing(board1)
x.solve()

number of iterations:  31097


31097

In [36]:
import Dataloader
import numpy as np

num = 100

data = Dataloader.make_dataset('sudoku.csv', limit=num)
iters = []

for sod in data:
    puzzle = sod[0]
    solver = SimulatedAnnealing(puzzle)
    iters.append( solver.solve())
    
print('Average number of iters was: ', np.mean(iters))
print('Median number of iters was: ', np.median(iters))
    


number of iterations:  29699
number of iterations:  32159
number of iterations:  29271
number of iterations:  29524
number of iterations:  29041
number of iterations:  43156
number of iterations:  24516
number of iterations:  30410
number of iterations:  31400
number of iterations:  42104
number of iterations:  27822
number of iterations:  27959
number of iterations:  36001
number of iterations:  28995
number of iterations:  31077
number of iterations:  38124
number of iterations:  30117
number of iterations:  31160
number of iterations:  26334
number of iterations:  39233
number of iterations:  33841
number of iterations:  148097
number of iterations:  16310
number of iterations:  34015
number of iterations:  31998
number of iterations:  29163
number of iterations:  29976
number of iterations:  150128
number of iterations:  30326
number of iterations:  6146457
number of iterations:  26209
number of iterations:  28224
number of iterations:  114789
number of iterations:  28869
number of

In [23]:
import Dataloader

data = Dataloader.make_dataset('sudoku.csv', limit=10)
solver = SimulatedAnnealing(data[4][0])
solver.solve()

Current Iteration:  1000 ; Current errors:  34 ; Current temperature:  9.049233858971451
Good swaps:  331 ; Bad swaps:  582 ; Rejected swaps:  86
Current Iteration:  2000 ; Current errors:  38 ; Current temperature:  8.188044457101192
Good swaps:  316 ; Bad swaps:  598 ; Rejected swaps:  86
Current Iteration:  3000 ; Current errors:  26 ; Current temperature:  7.408811958704974
Good swaps:  335 ; Bad swaps:  584 ; Rejected swaps:  81
Current Iteration:  4000 ; Current errors:  29 ; Current temperature:  6.7037367624262565
Good swaps:  319 ; Bad swaps:  580 ; Rejected swaps:  101
Current Iteration:  5000 ; Current errors:  30 ; Current temperature:  6.06576153240101
Good swaps:  331 ; Bad swaps:  586 ; Rejected swaps:  83
Current Iteration:  6000 ; Current errors:  37 ; Current temperature:  5.488500558998598
Good swaps:  334 ; Bad swaps:  574 ; Rejected swaps:  92
Current Iteration:  7000 ; Current errors:  29 ; Current temperature:  4.966175842096448
Good swaps:  324 ; Bad swaps:  578